In [ ]:
import requests
import datetime
import csv

# API Key
api_key = "YOUR_API_KEY"

# Base URL for traffic flow data
base_url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/{format}"

# Start and end points
start = "-6.935271,107.717554"
end = "-6.917358,107.574423"

# Date range
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=30)  # 30 days ago

# Open CSV file for writing
csv_file = "traffic_data.csv"
csv_columns = [
    "dateTime", "startPoint", "endPoint", "currentSpeed", "freeFlowSpeed",
    "currentTravelTime", "freeFlowTravelTime", "confidence", "roadClosure"
]

with open(csv_file, "w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()

    # Loop through dates
    current_date = start_date
    while current_date <= end_date:
        formatted_date = current_date.strftime("%Y-%m-%dT%H:%M:%S")
        request_url = f"{base_url}/{formatted_date}/json/{start}:{end}?key={api_key}"
        
        response = requests.get(request_url)
        data = response.json()

        if "flowSegmentData" in data:
            flow_data = data["flowSegmentData"]
            if flow_data:
                for segment in flow_data:
                    row = {
                        "dateTime": formatted_date,
                        "startPoint": start,
                        "endPoint": end,
                        "currentSpeed": segment.get("currentSpeed", ""),
                        "freeFlowSpeed": segment.get("freeFlowSpeed", ""),
                        "currentTravelTime": segment.get("currentTravelTime", ""),
                        "freeFlowTravelTime": segment.get("freeFlowTravelTime", ""),
                        "confidence": segment.get("confidence", ""),
                        "roadClosure": segment.get("roadClosure", "")
                    }
                    writer.writerow(row)
        current_date += datetime.timedelta(days=1)
        print(f"Processed data for {formatted_date}")

print("Data collection completed.")